# GIS Output CSV Merger
# version: M. Wong, 2019-08-11

Notes:<br>
(1) Python 3.6 Anaconda<br>
(2) Encoding UTF-8<br>
(3) Runs from directory with the following data structure<br>
> Ethnic_Background/2011_National_Household_Survey_Headers_Ethnic_Background.csv<br>
> Religious_Beliefs/Religious_Beliefs_Datum.csv<br>
> 07_08_2019_Project1_2_Canada_FSA_Block_Pop.csv<br>

Merges demographics CSV data with GIS outputted area csv

## Initialization

In [2]:
#Load Modules
import os
import re
import pandas as pd
import numpy as np

#Set pandas to display full column widths and full column numbers
#pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

print(pd.__version__)

0.24.2


In [46]:
## Check data
area_data = pd.read_csv("07_08_2019_Project1_2_Canada_FSA_Block_Pop.csv", delimiter=",")
area_data.drop(columns=["TARGET_DATA_POP_FSA_ABORIGINAL","CLIPPED_PROJECTED_BLACK_RAW_BLACK","CLIPPED_PROJECTED_CHINESE_RAW_CHINESE","CLIPPED_PROJECTED_LATIN_AMERICAN_RAW_LATIN_AMERICAN"], inplace=True)
area_data.rename(columns={"TARGET_DATA_POP_FSA_SPATIAL_ID":"FSA name"}, inplace=True)
display(area_data.head())
display(area_data.tail())
print(len(area_data))

,XCoord,YCoord,OBJECTID,FSA name,FSA_PRNAME,PAGENAME,SHAPE_LENGTH,SHAPE_AREA
0,95114.470706,1.026157e+06,1,P0W,Ontario,EOP2412,2614.927346,373658.978748
1,95817.766250,1.026228e+06,2,P0W,Ontario,EOP2413,2269.505728,123157.151841
2,96932.074631,1.026238e+06,3,P0W,Ontario,EOP2414,2756.701628,226353.595028
3,98185.809973,1.026401e+06,4,P0W,Ontario,EOP2415,990.932549,24270.572883
4,99101.926821,1.026216e+06,5,P0W,Ontario,EOP2416,3302.233512,466709.088671


,XCoord,YCoord,OBJECTID,FSA name,FSA_PRNAME,PAGENAME,SHAPE_LENGTH,SHAPE_AREA
10186776,670045.145400,2.553439e+06,10186777,X0C,Nunavut,CHW2987,4000.000000,1000000.000000
10186777,671045.145400,2.553439e+06,10186778,X0C,Nunavut,CHW2988,4000.000000,1000000.000000
10186778,672045.145400,2.553439e+06,10186779,X0C,Nunavut,CHW2989,4000.000000,1000000.000000
10186779,673045.145400,2.553439e+06,10186780,X0C,Nunavut,CHW2990,4000.000000,1000000.000000
10186780,673912.101204,2.553489e+06,10186781,X0C,Nunavut,CHW2991,3400.115004,711573.268031


10186781


In [47]:
## Some data checks
display(area_data[area_data["PAGENAME"] == "EOP2414"])

,XCoord,YCoord,OBJECTID,FSA name,FSA_PRNAME,PAGENAME,SHAPE_LENGTH,SHAPE_AREA
2,96932.074631,1.026238e+06,3,P0W,Ontario,EOP2414,2756.701628,226353.595028
38908,97026.251708,1.026680e+06,38909,P0X,Ontario,EOP2414,3658.496155,488393.146489


In [48]:
## Find area totals of each block
area_data["BLOCK_AREA"] = area_data.groupby(["PAGENAME"])["SHAPE_AREA"].transform("sum")
area_data.sort_values(by=["PAGENAME"],inplace=True)

In [49]:
display(area_data[area_data["PAGENAME"] == "EOP2413"])
display(area_data[area_data["PAGENAME"] == "EOP2414"])
display(area_data[area_data["PAGENAME"].str.contains("EOP241")])

,XCoord,YCoord,OBJECTID,FSA name,FSA_PRNAME,PAGENAME,SHAPE_LENGTH,SHAPE_AREA,BLOCK_AREA
38907,95991.357233,1.026679e+06,38908,P0X,Ontario,EOP2413,4730.176820,265248.682068,388405.833909
1,95817.766250,1.026228e+06,2,P0W,Ontario,EOP2413,2269.505728,123157.151841,388405.833909


,XCoord,YCoord,OBJECTID,FSA name,FSA_PRNAME,PAGENAME,SHAPE_LENGTH,SHAPE_AREA,BLOCK_AREA
2,96932.074631,1.026238e+06,3,P0W,Ontario,EOP2414,2756.701628,226353.595028,714746.741517
38908,97026.251708,1.026680e+06,38909,P0X,Ontario,EOP2414,3658.496155,488393.146489,714746.741517


,XCoord,YCoord,OBJECTID,FSA name,FSA_PRNAME,PAGENAME,SHAPE_LENGTH,SHAPE_AREA,BLOCK_AREA
38904,93045.145400,1.026439e+06,38905,P0X,Ontario,EOP2410,4000.000000,1000000.000000,1000000.000000
38905,93773.015878,1.026402e+06,38906,P0X,Ontario,EOP2411,4250.635171,392700.676370,392700.676370
38906,95029.479562,1.026606e+06,38907,P0X,Ontario,EOP2412,3839.439008,587348.907373,961007.886121
0,95114.470706,1.026157e+06,1,P0W,Ontario,EOP2412,2614.927346,373658.978748,961007.886121
38907,95991.357233,1.026679e+06,38908,P0X,Ontario,EOP2413,4730.176820,265248.682068,388405.833909
1,95817.766250,1.026228e+06,2,P0W,Ontario,EOP2413,2269.505728,123157.151841,388405.833909
2,96932.074631,1.026238e+06,3,P0W,Ontario,EOP2414,2756.701628,226353.595028,714746.741517
38908,97026.251708,1.026680e+06,38909,P0X,Ontario,EOP2414,3658.496155,488393.146489,714746.741517
38909,97962.423237,1.026736e+06,38910,P0X,Ontario,EOP2415,3533.733687,344412.598042,368683.170925
3,98185.809973,1.026401e+06,4,P0W,Ontario,EOP2415,990.932549,24270.572883,368683.170925


In [55]:
ethnic_data = pd.read_csv("Ethnic_Background/2011_National_Household_Survey_Headers_Ethnic_Background.csv", delimiter=",")
display(ethnic_data.head())

,COL0,COL1,COL2,COL3,COL4,COL5,COL6,COL7,COL8,COL9,COL10,COL11,COL12,COL13,COL14,COL15,COL16,COL17,COL18,COL19,COL20,COL21,COL22,COL23,COL24,COL25,COL26,COL27,COL28,COL29,COL30,COL31,COL32,COL33,COL34,COL35,COL36,COL37,COL38,COL39,COL40,COL41,COL42,COL43,COL44,COL45,COL46,COL47,COL48,COL49,COL50,COL51,COL52,COL53,COL54,COL55,COL56,COL57,COL58,COL59,COL60,COL61,COL62,COL63,COL64,COL65,COL66,COL67,COL68,COL69,COL70,COL71,COL72,COL73,COL74,COL75,COL76,COL77,COL78,COL79,COL80,COL81,COL82,COL83,COL84,COL85,COL86,COL87,COL88,COL89,COL90,COL91,COL92,COL93,COL94,COL95,COL96,COL97,COL98,COL99,COL100,COL101,COL102,COL103,COL104,COL105,COL106,COL107,COL108,COL109,COL110,COL111,COL112,COL113,COL114,COL115,COL116,COL117,COL118,COL119,COL120,COL121,COL122,COL123,COL124,COL125,COL126,COL127,COL128,COL129,COL130,COL131,COL132,COL133,COL134,COL135,COL136,COL137,COL138,COL139,COL140,COL141,COL142,COL143,COL144,COL145,COL146,COL147,COL148,COL149,COL150,COL151,COL152,COL153,COL154,COL155,COL156,COL157,COL158,COL159,COL160,COL161,COL162,COL163,COL164,COL165,COL166,COL167,COL168,COL169,COL170,COL171,COL172,COL173,COL174,COL175,COL176,COL177,COL178,COL179,COL180,COL181,COL182,COL183,COL184,COL185,COL186,COL187,COL188,COL189,COL190,COL191,COL192,COL193,COL194,COL195,COL196,COL197,COL198,COL199,COL200,COL201,COL202,COL203,COL204,COL205,COL206,COL207,COL208,COL209,COL210,COL211,COL212,COL213,COL214,COL215,COL216,COL217,COL218,COL219,COL220,COL221,COL222,COL223,COL224,COL225,COL226,COL227,COL228,COL229,COL230,COL231,COL232,COL233,COL234,COL235,COL236,COL237,COL238,COL239,COL240,COL241,COL242,COL243,COL244,COL245,COL246,COL247,COL248,COL249,COL250,COL251,COL252,COL253,COL254,COL255,COL256,COL257,COL258,COL259,COL260,COL261,COL262,COL263,COL264,COL265
0,32006402,A0A,45745.0,790.0,520.0,85.0,250.0,27415.0,0.0,185.0,26925.0,0.0,435.0,0.0,0.0,0.0,0.0,23915.0,23265.0,120.0,0.0,15515.0,11620.0,0.0,2155.0,180.0,255.0,1285.0,0.0,0.0,1285.0,620.0,0.0,0.0,95.0,0.0,0.0,525.0,0.0,0.0,0.0,130.0,15.0,0.0,0.0,95.0,15.0,0.0,230.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,20.0,20.0,0.0,65.0,0.0,220.0,0.0,0.0,0.0,0.0,0.0,85.0,0.0,0.0,0.0,0.0,65.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,70.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,32006403,A0B,20835.0,380.0,240.0,110.0,90.0,9225.0,0.0,45.0,9025.0,0.0,115.0,0.0,0.0,0.0,0.0,13535.0,13305.0,0.0,0.0,9515.0,5265.0,0.0,1225.0,120.0,150.0,415.0,0.0,0.0,410.0,215.0,0.0,15.0,40.0,0.0,0.0,165.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,20.0,0.0,0.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0,0.0,0.0,0.0,35.0,0.0,105.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [56]:
## Read header file
with open("Ethnic_Background/2011 National Household Survey Headers.txt", "r", encoding="latin-1") as ethnic_headers_file:
    lines = ethnic_headers_file.read().splitlines()
    #print(lines)
    ethnic_headers_file.close()
    
ethnic_headers = []
i = 0
for line in lines:
    if i == 0:
        pass
    else:
        ethnic_header = line.split("- ",2)
        ethnic_header = ethnic_header[-1]
        ethnic_header = ethnic_header.split("/ ", 2)[-1]
        ethnic_headers.append(ethnic_header)
    i += 1

#print(ethnic_headers)

ethnic_data.columns = ethnic_headers
ethnic_data.head()

,GEO UID,FSA name,Total population in private households by ethnic origins; Both sexes,North American Aboriginal origins; Both sexes,North American Aboriginal origins; Both sexes / First Nations (North American Indian); Both sexes,North American Aboriginal origins; Both sexes / Inuit; Both sexes,North American Aboriginal origins; Both sexes / Métis; Both sexes,Other North American origins; Both sexes,Other North American origins; Both sexes / Acadian; Both sexes,Other North American origins; Both sexes / American; Both sexes,Other North American origins; Both sexes / Canadian; Both sexes,Other North American origins; Both sexes / New Brunswicker; Both sexes,Other North American origins; Both sexes / Newfoundlander; Both sexes,Other North American origins; Both sexes / Nova Scotian; Both sexes,Other North American origins; Both sexes / Ontarian; Both sexes,Other North American origins; Both sexes / Québécois; Both sexes,Other North American origins; Both sexes / Other North American origins n.i.e.; Both sexes,European origins; Both sexes,European origins; Both sexes / British Isles origins; Both sexes,European origins; Both sexes / British Isles origins; Both sexes / Channel Islander; Both sexes,European origins; Both sexes / British Isles origins; Both sexes / Cornish; Both sexes,European origins; Both sexes / British Isles origins; Both sexes / English; Both sexes,European origins; Both sexes / British Isles origins; Both sexes / Irish; Both sexes,European origins; Both sexes / British Isles origins; Both sexes / Manx; Both sexes,European origins; Both sexes / British Isles origins; Both sexes / Scottish; Both sexes,European origins; Both sexes / British Isles origins; Both sexes / Welsh; Both sexes,European origins; Both sexes / British Isles origins; Both sexes / British Isles origins n.i.e.; Both sexes,European origins; Both sexes / French origins; Both sexes,European origins; Both sexes / French origins; Both sexes / Alsatian; Both sexes,European origins; Both sexes / French origins; Both sexes / Breton; Both sexes,European origins; Both sexes / French origins; Both sexes / French; Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes / Austrian; Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes / Belgian; Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes / Dutch; Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes / Flemish; Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes / Frisian; Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes / German; Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes / Luxembourger; Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes / Swiss; Both sexes,European origins; Both sexes / Western European origins (except French origins); Both sexes / Western European origins n.i.e.; Both sexes,European origins; Both sexes / Northern European origins (except British Isles origins); Both sexes,European origins; Both sexes / Northern European origins (except British Isles origins); Both sexes / Danish; Both sexes,European origins; Both sexes / Northern European origins (except British Isles origins); Both sexes / Finnish; Both sexes,European origins; Both sexes / Northern European origins (except British Isles origins); Both sexes / Icelandic; Both sexes,European origins; Both sexes / Northern European origins (except British Isles origins); Both sexes / Norwegian; Both sexes,European origins; Both sexes / Northern European origins (except British Isles origins); Both sexes / Swedish; Both sexes,European 

In [58]:
## Join ethnic dataframe to GIS dataframe on FSA
area_data = area_data.join(ethnic_data.set_index("FSA name"), on="FSA name")

In [ ]:
display(area_data[area_data["PAGENAME"] == "EOP2413"])

In [7]:
religion_data = pd.read_csv("Religious_Beliefs/Religious_Beliefs_Datum.csv", delimiter=",")
display(religion_data.head())

,COL0,COL1,COL2,COL3,COL4,COL5,COL6,COL7,COL8,COL9,COL10,COL11,COL12,COL13,COL14,COL15,COL16,COL17,COL18,COL19,COL20
0,32006402,A0A,45745.0,60.0,44195.0,12275.0,50.0,21635.0,15.0,30.0,1400.0,35.0,7055.0,1705.0,0.0,0.0,0.0,0.0,0.0,20.0,1470.0
1,32006403,A0B,20840.0,0.0,20230.0,5470.0,0.0,8580.0,0.0,0.0,425.0,10.0,3610.0,2110.0,0.0,0.0,35.0,0.0,0.0,0.0,530.0
2,32006404,A0C,12685.0,0.0,12425.0,4145.0,0.0,1550.0,0.0,0.0,695.0,15.0,4085.0,1930.0,0.0,0.0,0.0,0.0,0.0,0.0,255.0
3,32006405,A0E,23115.0,0.0,22720.0,5310.0,65.0,9835.0,0.0,15.0,660.0,0.0,4715.0,2115.0,0.0,0.0,25.0,0.0,0.0,0.0,365.0
4,32006406,A0G,35840.0,0.0,34410.0,8190.0,40.0,2930.0,25.0,0.0,6645.0,30.0,8860.0,7685.0,0.0,0.0,0.0,0.0,0.0,0.0,1410.0


In [8]:
## Read header file
with open("Religious_Beliefs/Religious_Beliefs_Headers.txt", "r", encoding="latin-1") as religion_headers_file:
    lines = religion_headers_file.read().splitlines()
    #print(lines)
    religion_headers_file.close()
    
religion_headers = []
i = 0
for line in lines:
    if i == 0:
        pass
    else:
        religion_header = line.split("- ",2)[-1]
        religion_header = religion_header.split("/ ", 2)[-1]
        religion_headers.append(religion_header)
    i += 1

#print(ethnic_headers)

religion_data.columns = religion_headers
religion_data.head()

,GEO UID,FSA name,Total population in private households by religion; Both sexes,Buddhist; Both sexes,Christian; Both sexes,Christian; Both sexes / Anglican; Both sexes,Christian; Both sexes / Baptist; Both sexes,Christian; Both sexes / Catholic; Both sexes,Christian; Both sexes / Christian Orthodox; Both sexes,Christian; Both sexes / Lutheran; Both sexes,Christian; Both sexes / Pentecostal; Both sexes,Christian; Both sexes / Presbyterian; Both sexes,Christian; Both sexes / United Church; Both sexes,Christian; Both sexes / Other Christian; Both sexes,Hindu; Both sexes,Jewish; Both sexes,Muslim; Both sexes,Sikh; Both sexes,Traditional (Aboriginal) Spirituality; Both sexes,Other religions; Both sexes,No religious affiliation; Both sexes
0,32006402,A0A,45745.0,60.0,44195.0,12275.0,50.0,21635.0,15.0,30.0,1400.0,35.0,7055.0,1705.0,0.0,0.0,0.0,0.0,0.0,20.0,1470.0
1,32006403,A0B,20840.0,0.0,20230.0,5470.0,0.0,8580.0,0.0,0.0,425.0,10.0,3610.0,2110.0,0.0,0.0,35.0,0.0,0.0,0.0,530.0
2,32006404,A0C,12685.0,0.0,12425.0,4145.0,0.0,1550.0,0.0,0.0,695.0,15.0,4085.0,1930.0,0.0,0.0,0.0,0.0,0.0,0.0,255.0
3,32006405,A0E,23115.0,0.0,22720.0,5310.0,65.0,9835.0,0.0,15.0,660.0,0.0,4715.0,2115.0,0.0,0.0,25.0,0.0,0.0,0.0,365.0
4,32006406,A0G,35840.0,0.0,34410.0,8190.0,40.0,2930.0,25.0,0.0,6645.0,30.0,8860.0,7685.0,0.0,0.0,0.0,0.0,0.0,0.0,1410.0


In [16]:
_filt.to_csv("lipad-house-senate-doc2vec-42nd-speeches.csv",index=False,encoding="utf-8")